In [1]:
import ipywidgets as widgets 
from ipywidgets import interact, interact_manual 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [8]:
data = pd.read_csv("../Data/preprocessingData/K_20200414_preprocessing15.csv",encoding='CP949')

In [9]:
data.head()

,date,(1101-1),(13-21),(145-0),(1617-21),(185-5),(191-8)중앙궁전빌라트,(20-3),(204-11),(246-11),...,힐타운,힐타워,힐탑,힐탑아파트3차(894-3),힐탑아파트4차(894-5),힐탑이루미,힐탑트레져1동,힐튼,힐튼빌리지1차,힐튼빌리지2차
0,2,692.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,327.0,NaN,345.0,NaN,NaN,NaN,NaN
1,3,754.0,481.0,NaN,NaN,NaN,420.0,NaN,NaN,NaN,...,NaN,NaN,344.0,277.0,NaN,354.0,NaN,293.0,NaN,NaN
2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,416.0,...,NaN,477.0,330.0,NaN,NaN,363.0,NaN,NaN,NaN,NaN
3,5,692.0,529.0,730.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,520.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,769.0,NaN,864.0,NaN,NaN,NaN,NaN,NaN,NaN,...,347.0,634.0,366.0,NaN,340.0,NaN,NaN,NaN,NaN,NaN
